In [92]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [93]:
%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import scipy as stats
from ast import literal_eval
from sklearn.feature_extraction.text import TfidfVectorizer,CountVectorizer
from sklearn.metrics.pairwise import linear_kernel, cosine_similarity
from nltk.stem.snowball import SnowballStemmer
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.corpus import wordnet


pd.pandas.set_option('display.max_columns',None)


from ast import literal_eval is used to evaluate the python literals like float,int,string etc.
eg: code = """(1, 2, {'foo': 'bar'})""" can be evaluated as Tuple when passed through the literal_eval object 


**Simmple Recomender**

In this we are going to recommend movies based on the gener and rating or popularity of movie watched by most people 

In [94]:
md = pd.read_csv('movies_metadata.csv')
md.head()

C:\Python 38\lib\site-packages\IPython\core\interactiveshell.py:3441: DtypeWarning: Columns (10) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,popularity,poster_path,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', 'poster_pa...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, 'name': 'Co...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his room until...",21.946943,/rhIRbceoE9lR4veEXuwCC2wARtG.jpg,"[{'name': 'Pixar Animation Studios', 'id': 3}]","[{'iso_3166_1': 'US', 'name': 'United States of America'}]",1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, 'name': 'Fa...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an enchanted board...,17.015539,/vzmL6fP7aPKNKPRTFnZmiUfciyV.jpg,"[{'name': 'TriStar Pictures', 'id': 559}, {'name': 'Teit...","[{'iso_3166_1': 'US', 'name': 'United States of America'}]",1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso_639_1': '...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collection', 'pos...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, 'name': 'C...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud between next...,11.7129,/6ksm1sjKMFLbO7UY2i6G1ju9SML.jpg,"[{'name': 'Warner Bros.', 'id': 6194}, {'name': 'Lancast...","[{'iso_3166_1': 'US', 'name': 'United States of America'}]",1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for Love.,Grumpier Old Men,False,6.5,92.0
3,False,NaN,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'name': 'Drama...",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the women are hol...",3.859495,/16XOMpEaLWkrcPqSQqhTmeJuqQl.jpg,"[{'name': 'Twentieth Century Fox Film Corporation', 'id'...","[{'iso_3166_1': 'US', 'name': 'United States of America'}]",1995-12-22,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself... and ne...,Waiting to Exhale,False,6.1,34.0
4,False,"{'id': 96871, 'name': 'Father of the Bride Collection', ...",0,"[{'id': 35, 'name': 'Comedy'}]",NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his daughter's...,8.387519,/e64sOI48hQXyru7naBFyssKFxVd.jpg,"[{'name': 'Sandollar Productions', 'id': 5842}, {'name':...","[{'iso_3166_1': 'US', 'name': 'United States of America'}]",1995-02-10,76578911.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's In For The...,Father of the Bride Part II,False,5.7,173.0


In [95]:
md['genres'] = md['genres'].fillna('[]').apply(literal_eval).apply(lambda x: [i['name'] for i in x]if isinstance(x,list) else [])

From the above we are taking out the genres values from the dictionary such that the name from the data alone is picked up and made into list. here literal_eval will evaluate the list we gave in the fillna and will apply the same.

TMBD Rating is used for Top Movie rating. we will be using IMDB Weighted Rating to construct the chart

Weighted Rating (WR) =  (vv+m.R)+(mv+m.C) 
where,

v is the number of votes for the movie
m is the minimum votes required to be listed in the chart
R is the average rating of the movie
C is the mean vote across the whole report

value of m is determined by taking 95th percentile as the cutoff. i.e we are taking movies having more votes than 95%

In [96]:
md['vote_count']

0        5415.0
1        2413.0
2          92.0
3          34.0
4         173.0
          ...  
45461       1.0
45462       3.0
45463       6.0
45464       0.0
45465       0.0
Name: vote_count, Length: 45466, dtype: float64

In [97]:
vote_count= md[md['vote_count'].notnull()]['vote_count'].astype('int')
vote_averages = md[md['vote_average'].notnull()]['vote_average'].astype('int')
C = vote_averages.mean()
C

5.244896612406511

In the above we are taking the notnull values alone and changeing the data type as 'int'

In [98]:
m = vote_count.quantile(0.95)
m

434.0

Above shows that for 95th Percentile the votecount of the movies must be above 434 and those movies are taken for consideration

In [99]:
md['release_date']

0        1995-10-30
1        1995-12-15
2        1995-12-22
3        1995-12-22
4        1995-02-10
            ...    
45461           NaN
45462    2011-11-17
45463    2003-08-01
45464    1917-10-21
45465    2017-06-09
Name: release_date, Length: 45466, dtype: object

In [100]:
pd.to_datetime(md['release_date'], errors='coerce')

0       1995-10-30
1       1995-12-15
2       1995-12-22
3       1995-12-22
4       1995-02-10
           ...    
45461          NaT
45462   2011-11-17
45463   2003-08-01
45464   1917-10-21
45465   2017-06-09
Name: release_date, Length: 45466, dtype: datetime64[ns]

In [101]:
md['year'] = pd.to_datetime(md['release_date'], errors='coerce').apply(lambda x: str(x).split('-')[0] if x != np.nan else np.nan)

The above will get the year from the release_date, to_datetime will convert the pandas datetime to datafram 

In [102]:
qualified = md[(md['vote_count'] >= m) & (md['vote_count'].notnull()) & (md['vote_average'].notnull())][['title', 'year', 'vote_count', 'vote_average', 'popularity', 'genres']]
qualified['vote_count'] = qualified['vote_count'].astype('int')
qualified['vote_average'] = qualified['vote_average'].astype('int')
qualified.shape

(2274, 6)

creating the new dataset containing the votecount graeter then the 95% percentile votecount ignoring the null values both from  vote_count and vote_average. and changing the type as int


we can see the vote count of the movie must be above 434, and the avarage rating will be 5.244896612406511

In [103]:
def weig_rat(x):
    v = x['vote_count']
    R = x['vote_average']
    return (v/(v+m)*R) + (m/(m+v)*C)

In [104]:
qualified['wr'] = qualified.apply(weig_rat,axis = 1)

In [105]:
qualified.head()

,title,year,vote_count,vote_average,popularity,genres,wr
0,Toy Story,1995,5415,7,21.946943,"[Animation, Comedy, Family]",6.869770
1,Jumanji,1995,2413,6,17.015539,"[Adventure, Fantasy, Family]",5.884891
5,Heat,1995,1886,7,17.924927,"[Action, Crime, Drama, Thriller]",6.671675
9,GoldenEye,1995,1194,6,14.686036,"[Adventure, Action, Thriller]",5.798701
15,Casino,1995,1343,7,10.137389,"[Drama, Crime]",6.571348


let's sort the values in decending order

In [106]:
qualified = qualified.sort_values('wr',ascending=False).head(250)

In [107]:
qualified.head(15)

,title,year,vote_count,vote_average,popularity,genres,wr
15480,Inception,2010,14075,8,29.108149,"[Action, Thriller, Science Fiction, Mystery, Adventure]",7.917588
12481,The Dark Knight,2008,12269,8,123.167259,"[Drama, Action, Crime, Thriller]",7.905871
22879,Interstellar,2014,11187,8,32.213481,"[Adventure, Drama, Science Fiction]",7.897107
2843,Fight Club,1999,9678,8,63.869599,[Drama],7.881753
4863,The Lord of the Rings: The Fellowship of the Ring,2001,8892,8,32.070725,"[Adventure, Fantasy, Action]",7.871787
292,Pulp Fiction,1994,8670,8,140.950236,"[Thriller, Crime]",7.868660
314,The Shawshank Redemption,1994,8358,8,51.645403,"[Drama, Crime]",7.864000
7000,The Lord of the Rings: The Return of the King,2003,8226,8,29.324358,"[Adventure, Fantasy, Action]",7.861927
351,Forrest Gump,1994,8147,8,48.307194,"[Comedy, Drama, Romance]",7.860656
5814,The Lord of the Rings: The Two Towers,2002,7641,8,29.423537,"[Adventure, Fantasy, Action]",7.851924


We can see that inception dark knight are having higher weighted rating which are christopher nolan film

In [108]:
s = md.apply(lambda x: pd.Series(x['genres']),axis=1).stack().reset_index(level=1, drop=True)

<ipython-input-108-0620b81dcd12>:1: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  s = md.apply(lambda x: pd.Series(x['genres']),axis=1).stack().reset_index(level=1, drop=True)


here we are taking the genres column and making it as a series the stack() Function will make the list of geners for movie as a stack and having the same index for the geners of the specific movie

In [109]:
s.name = 'genre'
gen_md = md.drop('genres', axis=1).join(s)

In [110]:
gen_md

,adult,belongs_to_collection,budget,homepage,id,imdb_id,original_language,original_title,overview,popularity,poster_path,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count,year,genre
0,False,"{'id': 10194, 'name': 'Toy Story Collection', 'poster_pa...",30000000,http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his room until...",21.946943,/rhIRbceoE9lR4veEXuwCC2wARtG.jpg,"[{'name': 'Pixar Animation Studios', 'id': 3}]","[{'iso_3166_1': 'US', 'name': 'United States of America'}]",1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0,1995,Animation
0,False,"{'id': 10194, 'name': 'Toy Story Collection', 'poster_pa...",30000000,http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his room until...",21.946943,/rhIRbceoE9lR4veEXuwCC2wARtG.jpg,"[{'name': 'Pixar Animation Studios', 'id': 3}]","[{'iso_3166_1': 'US', 'name': 'United States of America'}]",1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0,1995,Comedy
0,False,"{'id': 10194, 'name': 'Toy Story Collection', 'poster_pa...",30000000,http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his room until...",21.946943,/rhIRbceoE9lR4veEXuwCC2wARtG.jpg,"[{'name': 'Pixar Animation Studios', 'id': 3}]","[{'iso_3166_1': 'US', 'name': 'United States of America'}]",1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0,1995,Family
1,False,NaN,65000000,NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an enchanted board...,17.015539,/vzmL6fP7aPKNKPRTFnZmiUfciyV.jpg,"[{'name': 'TriStar Pictures', 'id': 559}, {'name': 'Teit...","[{'iso_3166_1': 'US', 'name': 'United States of America'}]",1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso_639_1': '...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0,1995,Adventure
1,False,NaN,65000000,NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an enchanted board...,17.015539,/vzmL6fP7aPKNKPRTFnZmiUfciyV.jpg,"[{'name': 'TriStar Pictures', 'id': 559}, {'name': 'Teit...","[{'iso_3166_1': 'US', 'name': 'United States of America'}]",1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso_639_1': '...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0,1995,Fantasy
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45463,False,NaN,0,NaN,67758,tt0303758,en,Betrayal,"When one of her hits goes wrong, a professional assassin...",0.903007,/d5bX92nDsISNhu3ZT69uHwmfCGw.jpg,"[{'name': 'American World Pictures', 'id': 6165}]","[{'iso_3166_1': 'US', 'name': 'United States of America'}]",2003-08-01,0.0,90.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,A deadly game of wits.,Betrayal,False,3.8,6.0,2003,Action
45463,False,NaN,0,NaN,67758,tt0303758,en,Betrayal,"When one of her hits goes wrong, a professional assassin...",0.903007,/d5bX92nDsISNhu3ZT69uHwmfCGw.jpg,"[{'name': 'American World Pictures', 'id': 6165}]","[{'iso_3166_1': 'US', 'name': 'United States of America'}]",2003-08-01,0.0,90.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,A deadly game of wits.,Betrayal,False,3.8,6.0,2003,Drama
45463,False,NaN,0,NaN,67758,tt0303758,en,Betrayal,"When one of her hits goes wrong, a professional assassin...",0.903007,/d5bX92nDsISNhu3ZT69uHwmfCGw.jpg,"[{'name': 'American World Pictures', 'id': 6165}]","[{'iso_3166_1': 'US', 'name': 'United States of America'}]",2003-08-01,0.0,90.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,A deadly game of wits.,Betrayal,False,3.8,6.0,2003,Thriller
45464,False,NaN,0,NaN,227506,tt0008536,en,Satana likuyushchiy,"In a sma

from the above we can see that we are droping the geners and concatinating the stacked seperated geners and get assingned to the sepecific movies

We have already calculated the weighted rating for the entire genres  columns for the movie having the vote count above 95% precentile. from that we found the most good rated movies. similarly  since we seperated the movies in tearm of genres we are going to create a function which finds the top rated movie based on specific genres. and we are going to take 85% as the criticle value of vote count. so the movies above 85% of vote count of total vote count wiill be selected.

In [111]:
def top_movie(genre,n_precentile =.85):
    df = gen_md[gen_md['genre'] == genre]
    vote_count = df[df['vote_count'].notnull()]['vote_count'].astype('int')
    vote_avg = df[df['vote_average'].notnull()]['vote_average'].astype('int')
    C = vote_avg.mean()
    m = vote_count.quantile(n_precentile)
    qualified = df[(df['vote_count']>= m) & (df['vote_count'].notnull()) & (df['vote_average'].notnull())][['title', 'year', 'vote_count', 'vote_average', 'popularity']]
    qualified['vote_count'] = qualified['vote_count'].astype('int')
    qualified['vote_average'] = qualified['vote_average'].astype('int')
    qualified['wr'] = qualified.apply(lambda x: (x['vote_count']/(x['vote_count']+m)*x['vote_average'])+(m/(m+x['vote_count'])*C),axis = 1)
    qualified = qualified.sort_values('wr',ascending =False).head(250)
    return qualified
    

In [112]:
top_movie('Action')

,title,year,vote_count,vote_average,popularity,wr
15480,Inception,2010,14075,8,29.108149,7.955099
12481,The Dark Knight,2008,12269,8,123.167259,7.948610
4863,The Lord of the Rings: The Fellowship of the Ring,2001,8892,8,32.070725,7.929579
7000,The Lord of the Rings: The Return of the King,2003,8226,8,29.324358,7.924031
5814,The Lord of the Rings: The Two Towers,2002,7641,8,29.423537,7.918382
...,...,...,...,...,...,...
21296,The Wolverine,2013,4110,6,3.918287,5.956636
13635,X-Men Origins: Wolverine,2009,4086,6,1.456541,5.956395
5244,Star Wars: Episode II - Attack of the Clones,2002,4074,6,14.072511,5.956273
18289,Mission: Impossible - Ghost Protocol,2011,4026,6,14.25843,5.955779


### Content Based Recommender

As of above, we have created simple recommender system where we can get the movies recommended based on the ratings and based on the genre. But with this recommendation system we cannot able to recommend a movie based on personal choice.

for example if you consider Inception the Darknight are the movie by cristopher nolan from this we can make a assumption that the person Like Cristopher nolan movies.But he can only see the action movies. where most of them were made by differnt directors and not by cristopher nolan

To personalise this recommendation we need some more features to form a certain metric to recommend the movie

we are going to build two content based system

> Movie Overviews and Taglines

> Movie Cast, Crew, Keywords and Genre

In [113]:
link_small = pd.read_csv('links_small.csv')
link_small.head()

,movieId,imdbId,tmdbId
0,1,114709,862.0
1,2,113497,8844.0
2,3,113228,15602.0
3,4,114885,31357.0
4,5,113041,11862.0


In [114]:
print(link_small['tmdbId'].isnull().sum())
print(link_small.info())

13
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9125 entries, 0 to 9124
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   movieId  9125 non-null   int64  
 1   imdbId   9125 non-null   int64  
 2   tmdbId   9112 non-null   float64
dtypes: float64(1), int64(2)
memory usage: 214.0 KB
None


we have 13 null values removing those and changing tmdbId to int64

0          862
1         8844
2        15602
3        31357
4        11862
         ...  
9120    402672
9121    315011
9122    391698
9123    137608
9124    410803
Name: tmdbId, Length: 9112, dtype: int32

In [29]:
md.isnull().sum()

adult                        0
belongs_to_collection    40972
budget                       0
genres                       0
homepage                 37684
id                           0
imdb_id                     17
original_language           11
original_title               0
overview                   954
popularity                   5
poster_path                386
production_companies         3
production_countries         3
release_date                87
revenue                      6
runtime                    263
spoken_languages             6
status                      87
tagline                  25054
title                        6
video                        6
vote_average                 6
vote_count                   6
year                         0
dtype: int64

In [30]:
md['id']

0           862
1          8844
2         15602
3         31357
4         11862
          ...  
45461    439050
45462    111109
45463     67758
45464    227506
45465    461257
Name: id, Length: 45466, dtype: object

In [31]:
md = md.drop([19730, 29503, 35587])

Droping worngly indexed row found by feature analysis

In [32]:
md['id'] = md['id'].astype('int')

In [33]:
smd = md[md['id'].isin(link_small)]
smd.shape

(9099, 25)

We have 9099 movies got by modeling the dataset or joining the table which is 5 times smaller than original dataset

#### Movie Discription Based Recommender





In [34]:
smd['tagline'] = smd['tagline'].fillna('')

<ipython-input-34-5268a1d163a5>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  smd['tagline'] = smd['tagline'].fillna('')


we can see using tag line we can work out some combinations

In [35]:
smd['description'] = smd['overview'] + smd['tagline']


<ipython-input-35-362f3d9a1c27>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  smd['description'] = smd['overview'] + smd['tagline']


we combined overview and tagline to encode and find some similarity 

In [36]:
smd['description'] = smd['description'].fillna('')

<ipython-input-36-794954ed571a>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  smd['description'] = smd['description'].fillna('')


using TFid vectoriser to encode the description

In [37]:
from sklearn.feature_extraction.text import TfidfVectorizer

tf = TfidfVectorizer(analyzer='word',ngram_range=(1,2),min_df=0,stop_words='english')
tfid_mat = tf.fit_transform(smd['description'])

here analyzer indicates what to be analysed in the document like word, number,characters

ngram_range is to specify the grouping in the tfid vectoriser (1,2) indicates group the letters in unigram and bigram. where if you consider a document 'i am studying NLP' for bigram this will group as 'i am' 'am studying' 'studying NLP'

thus encoding the descrption in TFid matrix

we will use Cosine similarity to find the similarity between the movies. such that cosine similarity is understood by ploting the data in the x y plot,where x and y are the each document tfid values. since the dot product of the cosine similarity is alredy found we will use
linear_kernal to find the similarity

In [38]:
cos_sim = linear_kernel(tfid_mat,tfid_mat)

In [39]:
len(cos_sim[0])

9099

from the above we can see that each record created alist of values containing their similarity

In [40]:
smd = smd.reset_index()

In [41]:
titles = smd['title']

In [42]:
indices = pd.Series(smd.index,index=smd['title'])
indices

title
Toy Story                                                0
Jumanji                                                  1
Grumpier Old Men                                         2
Waiting to Exhale                                        3
Father of the Bride Part II                              4
                                                      ... 
Shin Godzilla                                         9094
The Beatles: Eight Days a Week - The Touring Years    9095
Pokémon: Spell of the Unknown                         9096
Pokémon 4Ever: Celebi - Voice of the Forest           9097
Force Majeure                                         9098
Length: 9099, dtype: int64

we made the title as index and index as values so that we can match the index of cosine similarity with the title.

In [43]:
idx = indices['The Dark Knight']

getting the index of the movie we need

In [44]:
sim_scores = list(enumerate(cos_sim[idx]))
sim_scores[:5]

[(0, 0.0), (1, 0.007774131635450035), (2, 0.0), (3, 0.0), (4, 0.0)]

enumarating the cosine similarity of all the movies based on the dark night raises to match the index

In [45]:
sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True) #sorting the movies based on the max similarity score
sim_scores = sim_scores[1:31]
movie_indices = [i[0] for i in sim_scores]#getting the similarity score of the movies based on movie 'the dark night raises'
print(titles.iloc[movie_indices])

7931                      The Dark Knight Rises
132                              Batman Forever
1113                             Batman Returns
8227    Batman: The Dark Knight Returns, Part 2
7565                 Batman: Under the Red Hood
524                                      Batman
7901                           Batman: Year One
2579               Batman: Mask of the Phantasm
2696                                        JFK
8165    Batman: The Dark Knight Returns, Part 1
6144                              Batman Begins
7933         Sherlock Holmes: A Game of Shadows
5511                            To End All Wars
4489                                      Q & A
7344                        Law Abiding Citizen
7242                  The File on Thelma Jordon
3537                               Criminal Law
2893                              Flying Tigers
1135                   Night Falls on Manhattan
8680                          The Young Savages
8917         Batman v Superman: Dawn of 

by printing we would get the movies similar to dark night raises based on the tagline and discription

### now creating a function

In [46]:
def get_recommendations(title):
    idx = indices[title]#Gettting the index of the movie
    sim_scores = list(enumerate(cos_sim[idx])) #finding the cos similarity of the movie using it index and enumarating the similarity
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True) #sorting the movie based on the similarity score 
    sim_scores = sim_scores[1:31] # taking first 30 movies
    movie_indices = [i[0] for i in sim_scores] # taking the sorted movies 
    return titles.iloc[movie_indices]

In [47]:
get_recommendations('The Godfather').head(10)

973      The Godfather: Part II
8387                 The Family
3509                       Made
4196         Johnny Dangerously
29               Shanghai Triad
5667                       Fury
2412             American Movie
1582    The Godfather: Part III
4221                    8 Women
2159              Summer of Sam
Name: title, dtype: object

In [48]:
get_recommendations('The Dark Knight').head(10)

7931                      The Dark Knight Rises
132                              Batman Forever
1113                             Batman Returns
8227    Batman: The Dark Knight Returns, Part 2
7565                 Batman: Under the Red Hood
524                                      Batman
7901                           Batman: Year One
2579               Batman: Mask of the Phantasm
2696                                        JFK
8165    Batman: The Dark Knight Returns, Part 1
Name: title, dtype: object

From the above we can able to see that when we give the dark night it gives all the batman movies . where it found the similarity based on discription and taglines. This recomendation are good when we want a sequel of the movie. but suppose if the movie doesn't have any sequel then we have to recommend movies based on other criteria like cast,crew,and geners and keywords.

In [49]:
credits = pd.read_csv('credits.csv')
keyword = pd.read_csv('keywords.csv')

In [50]:
pd.set_option('max_colwidth',60)
credits['cast'][0:1][0]

"[{'cast_id': 14, 'character': 'Woody (voice)', 'credit_id': '52fe4284c3a36847f8024f95', 'gender': 2, 'id': 31, 'name': 'Tom Hanks', 'order': 0, 'profile_path': '/pQFoyx7rp09CJTAb932F2g8Nlho.jpg'}, {'cast_id': 15, 'character': 'Buzz Lightyear (voice)', 'credit_id': '52fe4284c3a36847f8024f99', 'gender': 2, 'id': 12898, 'name': 'Tim Allen', 'order': 1, 'profile_path': '/uX2xVf6pMmPepxnvFWyBtjexzgY.jpg'}, {'cast_id': 16, 'character': 'Mr. Potato Head (voice)', 'credit_id': '52fe4284c3a36847f8024f9d', 'gender': 2, 'id': 7167, 'name': 'Don Rickles', 'order': 2, 'profile_path': '/h5BcaDMPRVLHLDzbQavec4xfSdt.jpg'}, {'cast_id': 17, 'character': 'Slinky Dog (voice)', 'credit_id': '52fe4284c3a36847f8024fa1', 'gender': 2, 'id': 12899, 'name': 'Jim Varney', 'order': 3, 'profile_path': '/eIo2jVVXYgjDtaHoF19Ll9vtW7h.jpg'}, {'cast_id': 18, 'character': 'Rex (voice)', 'credit_id': '52fe4284c3a36847f8024fa5', 'gender': 2, 'id': 12900, 'name': 'Wallace Shawn', 'order': 4, 'profile_path': '/oGE6JqPP2xH4t

In [51]:
keyword['id'] = keyword['id'].astype('int')
credits['id'] = credits['id'].astype('int')
md['id'] = md['id'].astype('int')

Changind the data type as Int

In [52]:
md = md.merge(keyword,on='id')


merging the keword dataset to md dataset based on its ID

In [53]:
md = md.merge(credits,on='id')


merging the credits dataset to md dataset based on its ID

In [54]:
smd = md[md['id'].isin(link_small)]
smd

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,popularity,poster_path,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count,year,keywords,cast,crew
0,False,"{'id': 10194, 'name': 'Toy Story Collection', 'poster_pa...",30000000,"[Animation, Comedy, Family]",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his room until...",21.946943,/rhIRbceoE9lR4veEXuwCC2wARtG.jpg,"[{'name': 'Pixar Animation Studios', 'id': 3}]","[{'iso_3166_1': 'US', 'name': 'United States of America'}]",1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0,1995,"[{'id': 931, 'name': 'jealousy'}, {'id': 4290, 'name': '...","[{'cast_id': 14, 'character': 'Woody (voice)', 'credit_i...","[{'credit_id': '52fe4284c3a36847f8024f49', 'department':..."
1,False,NaN,65000000,"[Adventure, Fantasy, Family]",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an enchanted board...,17.015539,/vzmL6fP7aPKNKPRTFnZmiUfciyV.jpg,"[{'name': 'TriStar Pictures', 'id': 559}, {'name': 'Teit...","[{'iso_3166_1': 'US', 'name': 'United States of America'}]",1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso_639_1': '...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0,1995,"[{'id': 10090, 'name': 'board game'}, {'id': 10941, 'nam...","[{'cast_id': 1, 'character': 'Alan Parrish', 'credit_id'...","[{'credit_id': '52fe44bfc3a36847f80a7cd1', 'department':..."
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collection', 'pos...",0,"[Romance, Comedy]",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud between next...,11.7129,/6ksm1sjKMFLbO7UY2i6G1ju9SML.jpg,"[{'name': 'Warner Bros.', 'id': 6194}, {'name': 'Lancast...","[{'iso_3166_1': 'US', 'name': 'United States of America'}]",1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for Love.,Grumpier Old Men,False,6.5,92.0,1995,"[{'id': 1495, 'name': 'fishing'}, {'id': 12392, 'name': ...","[{'cast_id': 2, 'character': 'Max Goldman', 'credit_id':...","[{'credit_id': '52fe466a9251416c75077a89', 'department':..."
3,False,NaN,16000000,"[Comedy, Drama, Romance]",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the women are hol...",3.859495,/16XOMpEaLWkrcPqSQqhTmeJuqQl.jpg,"[{'name': 'Twentieth Century Fox Film Corporation', 'id'...","[{'iso_3166_1': 'US', 'name': 'United States of America'}]",1995-12-22,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself... and ne...,Waiting to Exhale,False,6.1,34.0,1995,"[{'id': 818, 'name': 'based on novel'}, {'id': 10131, 'n...","[{'cast_id': 1, 'character': ""Savannah 'Vannah' Jackson""...","[{'credit_id': '52fe44779251416c91011acb', 'department':..."
4,False,"{'id': 96871, 'name': 'Father of the Bride Collection', ...",0,[Comedy],NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his daughter's...,8.387519,/e64sOI48hQXyru7naBFyssKFxVd.jpg,"[{'name': 'Sandollar Productions', 'id': 5842}, {'name':...","[{'iso_3166_1': 'US', 'name': 'United States of America'}]",1995-02-10,76578911.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's In For The...,Father of the Bride Part II,False,5.7,173.0,1995,"[{'id': 1009, 'name': 'baby'}, {'id': 1599, 'name': 'mid...","[{'cast_id': 1, 'character': 'George Banks', 'credit_id'...","[{'credit_id': '52fe44959251416c75039ed7', 'department':..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40952,False,NaN,8000000,[Drama],NaN,159550,tt0255313,en,The Last Brickmaker in America,A man must cope with the loss o

In [55]:
smd['cast'] = smd['cast'].apply(literal_eval)
smd['crew'] = smd['crew'].apply(literal_eval)
smd['keywords'] = smd['keywords'].apply(literal_eval)
smd['cast_size'] = smd['cast'].apply(lambda x: len(x))
smd['crew_size'] = smd['crew'].apply(lambda x: len(x))


<ipython-input-55-0380b14b24f8>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  smd['cast'] = smd['cast'].apply(literal_eval)
<ipython-input-55-0380b14b24f8>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  smd['crew'] = smd['crew'].apply(literal_eval)
<ipython-input-55-0380b14b24f8>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-doc

we use literal_eval to find the dictionary,list or tuple kind  of datas. which will evaluate the list data as single data. because of this we can able to find the length of dic inside  the list.     

In [56]:
print(smd['cast'][0:1][0])# first row cast details
print(len(smd['cast'][0:1][0])) #length  of the first row cast



[{'cast_id': 14, 'character': 'Woody (voice)', 'credit_id': '52fe4284c3a36847f8024f95', 'gender': 2, 'id': 31, 'name': 'Tom Hanks', 'order': 0, 'profile_path': '/pQFoyx7rp09CJTAb932F2g8Nlho.jpg'}, {'cast_id': 15, 'character': 'Buzz Lightyear (voice)', 'credit_id': '52fe4284c3a36847f8024f99', 'gender': 2, 'id': 12898, 'name': 'Tim Allen', 'order': 1, 'profile_path': '/uX2xVf6pMmPepxnvFWyBtjexzgY.jpg'}, {'cast_id': 16, 'character': 'Mr. Potato Head (voice)', 'credit_id': '52fe4284c3a36847f8024f9d', 'gender': 2, 'id': 7167, 'name': 'Don Rickles', 'order': 2, 'profile_path': '/h5BcaDMPRVLHLDzbQavec4xfSdt.jpg'}, {'cast_id': 17, 'character': 'Slinky Dog (voice)', 'credit_id': '52fe4284c3a36847f8024fa1', 'gender': 2, 'id': 12899, 'name': 'Jim Varney', 'order': 3, 'profile_path': '/eIo2jVVXYgjDtaHoF19Ll9vtW7h.jpg'}, {'cast_id': 18, 'character': 'Rex (voice)', 'credit_id': '52fe4284c3a36847f8024fa5', 'gender': 2, 'id': 12900, 'name': 'Wallace Shawn', 'order': 4, 'profile_path': '/oGE6JqPP2xH4tN

from the above literal_eval library is used to evaluate non string data types in a list. therefore it consider the list of dictionary which helps us to count the length of the cast

In [57]:
def direct(na):
    for i in na:
        if i['job'] == 'Director':
            return i['name']
    return np.nan

taking the director name from the data set 

In [58]:
smd['directors'] = smd['crew'].apply(direct)

<ipython-input-58-80a5d46d31b7>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  smd['directors'] = smd['crew'].apply(direct)


In [59]:
smd['cast']

0        [{'cast_id': 14, 'character': 'Woody (voice)', 'credit_i...
1        [{'cast_id': 1, 'character': 'Alan Parrish', 'credit_id'...
2        [{'cast_id': 2, 'character': 'Max Goldman', 'credit_id':...
3        [{'cast_id': 1, 'character': 'Savannah 'Vannah' Jackson'...
4        [{'cast_id': 1, 'character': 'George Banks', 'credit_id'...
                                    ...                             
40952    [{'cast_id': 1, 'character': 'Henry Cobb', 'credit_id': ...
41172    [{'cast_id': 0, 'character': 'Rustom Pavri', 'credit_id'...
41225    [{'cast_id': 0, 'character': 'Sarman', 'credit_id': '576...
41391    [{'cast_id': 4, 'character': 'Rando Yaguchi : Deputy Chi...
41669    [{'cast_id': 0, 'character': 'Himself', 'credit_id': '57...
Name: cast, Length: 9219, dtype: object

In [60]:
smd['cast'] = smd['cast'].apply(lambda x: [i['name'] for i in x] if isinstance(x, list) else [])
smd['cast'] = smd['cast'].apply(lambda x: x[:3] if len(x) >=3 else x)

<ipython-input-60-e8ba85dfed9e>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  smd['cast'] = smd['cast'].apply(lambda x: [i['name'] for i in x] if isinstance(x, list) else [])
<ipython-input-60-e8ba85dfed9e>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  smd['cast'] = smd['cast'].apply(lambda x: x[:3] if len(x) >=3 else x)


taking th character names alone and making it as cast then taking first three cast members

In [61]:
smd['cast']

0                          [Tom Hanks, Tim Allen, Don Rickles]
1               [Robin Williams, Jonathan Hyde, Kirsten Dunst]
2                   [Walter Matthau, Jack Lemmon, Ann-Margret]
3            [Whitney Houston, Angela Bassett, Loretta Devine]
4                   [Steve Martin, Diane Keaton, Martin Short]
                                 ...                          
40952          [Sidney Poitier, Wendy Crewson, Jay O. Sanders]
41172                [Akshay Kumar, Ileana D'Cruz, Esha Gupta]
41225                [Hrithik Roshan, Pooja Hegde, Kabir Bedi]
41391    [Hiroki Hasegawa, Yutaka Takenouchi, Satomi Ishihara]
41669               [Paul McCartney, Ringo Starr, John Lennon]
Name: cast, Length: 9219, dtype: object

In [62]:
smd['keywords'] = smd['keywords'].apply(lambda x: [ i['name'] for i in x] if isinstance(x,list) else [])

<ipython-input-62-19bff800a09f>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  smd['keywords'] = smd['keywords'].apply(lambda x: [ i['name'] for i in x] if isinstance(x,list) else [])


In [63]:
smd['keywords']

0        [jealousy, toy, boy, friendship, friends, rivalry, boy n...
1        [board game, disappearance, based on children's book, ne...
2              [fishing, best friend, duringcreditsstinger, old men]
3        [based on novel, interracial relationship, single mother...
4        [baby, midlife crisis, confidence, aging, daughter, moth...
                                    ...                             
40952                                     [friendship, brick making]
41172                                                    [bollywood]
41225                                                    [bollywood]
41391    [monster, godzilla, giant monster, destruction, kaiju, t...
41669                                           [music, documentary]
Name: keywords, Length: 9219, dtype: object

Similarly as we did recommendation based on description we will be using metadata features like crew, directors, cast and combaining all the data then doing count vectorisation. 

In [64]:
smd['cast'] = smd['cast'].apply(lambda x: [str.lower(i.replace(" ","")) for i in x])

<ipython-input-64-566edfad31bc>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  smd['cast'] = smd['cast'].apply(lambda x: [str.lower(i.replace(" ","")) for i in x])


In [65]:
smd['cast']

0                          [tomhanks, timallen, donrickles]
1               [robinwilliams, jonathanhyde, kirstendunst]
2                  [waltermatthau, jacklemmon, ann-margret]
3            [whitneyhouston, angelabassett, lorettadevine]
4                   [stevemartin, dianekeaton, martinshort]
                                ...                        
40952           [sidneypoitier, wendycrewson, jayo.sanders]
41172                [akshaykumar, ileanad'cruz, eshagupta]
41225                [hrithikroshan, poojahegde, kabirbedi]
41391    [hirokihasegawa, yutakatakenouchi, satomiishihara]
41669               [paulmccartney, ringostarr, johnlennon]
Name: cast, Length: 9219, dtype: object

In [66]:
smd.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9219 entries, 0 to 41669
Data columns (total 31 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   adult                  9219 non-null   object 
 1   belongs_to_collection  1688 non-null   object 
 2   budget                 9219 non-null   object 
 3   genres                 9219 non-null   object 
 4   homepage               2001 non-null   object 
 5   id                     9219 non-null   int32  
 6   imdb_id                9219 non-null   object 
 7   original_language      9219 non-null   object 
 8   original_title         9219 non-null   object 
 9   overview               9207 non-null   object 
 10  popularity             9219 non-null   object 
 11  poster_path            9216 non-null   object 
 12  production_companies   9219 non-null   object 
 13  production_countries   9219 non-null   object 
 14  release_date           9219 non-null   object 
 15  rev

In [67]:
smd['directors'] = smd['directors'].astype('str').apply(lambda x: str.lower(x.replace(" ","")))

<ipython-input-67-348eb8bef5e7>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  smd['directors'] = smd['directors'].astype('str').apply(lambda x: str.lower(x.replace(" ","")))


In [68]:
smd['directors'] =smd['directors'].apply(lambda x: [x,x, x])

<ipython-input-68-fb40811c6939>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  smd['directors'] =smd['directors'].apply(lambda x: [x,x, x])


In [69]:
smd['directors'][0]

['johnlasseter', 'johnlasseter', 'johnlasseter']

from the above we can see that we are making directors into three stacked list inorder to match the crew members

In [70]:
s = smd.apply(lambda x: pd.Series(x['keywords']),axis=1).stack().reset_index(level =1,drop = True)
s.name = 'keyword'

<ipython-input-70-e19616d03ac2>:1: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  s = smd.apply(lambda x: pd.Series(x['keywords']),axis=1).stack().reset_index(level =1,drop = True)


In [71]:
s = s.value_counts()
s = s[s>1]

taking the most important keywords corpus sepeartely and removing records having less keywords

In [72]:

def keywords(x):
    m = []
    for i in x:
        if i in s:
            m.append(i)
    return m

function to get the most importtant keywords and removing rest

In [73]:
stemmer = SnowballStemmer('english')
stemmer.stem('catching')

'catch'

above is the example of stemming

In [74]:
smd['keywords'] = smd['keywords'].apply(keywords)
smd['keywords'] = smd['keywords'].apply(lambda x: [stemmer.stem(i) for i in x])
smd['keywords'] = smd['keywords'].apply(lambda x: [str.lower(i.replace(" ","")) for i in x])
smd['keywords']

<ipython-input-74-cb0a107a9064>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  smd['keywords'] = smd['keywords'].apply(keywords)
<ipython-input-74-cb0a107a9064>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  smd['keywords'] = smd['keywords'].apply(lambda x: [stemmer.stem(i) for i in x])
<ipython-input-74-cb0a107a9064>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation

0        [jealousi, toy, boy, friendship, friend, rivalri, boynex...
1        [boardgam, disappear, basedonchildren'sbook, newhom, rec...
2                             [fish, bestfriend, duringcreditssting]
3        [basedonnovel, interracialrelationship, singlemoth, divo...
4        [babi, midlifecrisi, confid, age, daughter, motherdaught...
                                    ...                             
40952                                                   [friendship]
41172                                                    [bollywood]
41225                                                    [bollywood]
41391               [monster, godzilla, giantmonst, destruct, kaiju]
41669                                           [music, documentari]
Name: keywords, Length: 9219, dtype: object

In [75]:
smd['soup'] = smd['keywords'] + smd['cast'] + smd['directors'] + smd['genres']
smd['soup'][0]

<ipython-input-75-bda9b963d4a2>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  smd['soup'] = smd['keywords'] + smd['cast'] + smd['directors'] + smd['genres']


['jealousi',
 'toy',
 'boy',
 'friendship',
 'friend',
 'rivalri',
 'boynextdoor',
 'newtoy',
 'toycomestolif',
 'tomhanks',
 'timallen',
 'donrickles',
 'johnlasseter',
 'johnlasseter',
 'johnlasseter',
 'Animation',
 'Comedy',
 'Family']

combining all the key words directors,geners,cast,crew etc to get a list of words

In [76]:
smd['soup'] = smd['soup'].apply(lambda x: " ".join(x))

<ipython-input-76-53e2a72485cd>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  smd['soup'] = smd['soup'].apply(lambda x: " ".join(x))


In [90]:
smd

,index,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,popularity,poster_path,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count,year,keywords,cast,crew,cast_size,crew_size,directors,soup
0,0,False,"{'id': 10194, 'name': 'Toy Story Collection', 'poster_pa...",30000000,"[Animation, Comedy, Family]",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his room until...",21.946943,/rhIRbceoE9lR4veEXuwCC2wARtG.jpg,"[{'name': 'Pixar Animation Studios', 'id': 3}]","[{'iso_3166_1': 'US', 'name': 'United States of America'}]",1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0,1995,"[jealousi, toy, boy, friendship, friend, rivalri, boynex...","[tomhanks, timallen, donrickles]","[{'credit_id': '52fe4284c3a36847f8024f49', 'department':...",13,106,"[johnlasseter, johnlasseter, johnlasseter]",jealousi toy boy friendship friend rivalri boynextdoor n...
1,1,False,NaN,65000000,"[Adventure, Fantasy, Family]",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an enchanted board...,17.015539,/vzmL6fP7aPKNKPRTFnZmiUfciyV.jpg,"[{'name': 'TriStar Pictures', 'id': 559}, {'name': 'Teit...","[{'iso_3166_1': 'US', 'name': 'United States of America'}]",1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso_639_1': '...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0,1995,"[boardgam, disappear, basedonchildren'sbook, newhom, rec...","[robinwilliams, jonathanhyde, kirstendunst]","[{'credit_id': '52fe44bfc3a36847f80a7cd1', 'department':...",26,16,"[joejohnston, joejohnston, joejohnston]",boardgam disappear basedonchildren'sbook newhom reclus g...
2,2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collection', 'pos...",0,"[Romance, Comedy]",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud between next...,11.7129,/6ksm1sjKMFLbO7UY2i6G1ju9SML.jpg,"[{'name': 'Warner Bros.', 'id': 6194}, {'name': 'Lancast...","[{'iso_3166_1': 'US', 'name': 'United States of America'}]",1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for Love.,Grumpier Old Men,False,6.5,92.0,1995,"[fish, bestfriend, duringcreditssting]","[waltermatthau, jacklemmon, ann-margret]","[{'credit_id': '52fe466a9251416c75077a89', 'department':...",7,4,"[howarddeutch, howarddeutch, howarddeutch]",fish bestfriend duringcreditssting waltermatthau jacklem...
3,3,False,NaN,16000000,"[Comedy, Drama, Romance]",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the women are hol...",3.859495,/16XOMpEaLWkrcPqSQqhTmeJuqQl.jpg,"[{'name': 'Twentieth Century Fox Film Corporation', 'id'...","[{'iso_3166_1': 'US', 'name': 'United States of America'}]",1995-12-22,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself... and ne...,Waiting to Exhale,False,6.1,34.0,1995,"[basedonnovel, interracialrelationship, singlemoth, divo...","[whitneyhouston, angelabassett, lorettadevine]","[{'credit_id': '52fe44779251416c91011acb', 'department':...",10,10,"[forestwhitaker, forestwhitaker, forestwhitaker]",basedonnovel interracialrelationship singlemoth divorc c...
4,4,False,"{'id': 96871, 'name': 'Father of the Bride Collection', ...",0,[Comedy],NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his daughter's...,8.387519,/e64sOI48hQXyru7naBFyssKFxVd.jpg,"[{'name': 'Sandollar Productions', 'id': 5842}, {'name':...","[{'iso_3166_1': 'US', 'name': 'United States of America'}]",1995-02-10,76578911.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's In For The...,Father of the Bride Part II,False,5.7,173.0,1995,"[babi, midlifecrisi,

In [78]:
count = CountVectorizer(analyzer='word',ngram_range=(1,2),min_df=0,stop_words='english')
count_mat = count.fit_transform(smd['soup'])

In [79]:
cos_sim=cosine_similarity(count_mat,count_mat)

In [80]:
smd = smd.reset_index()
titles = smd['title']
indices = pd.Series(smd.index,index=smd['title'])
indices

title
Toy Story                                                0
Jumanji                                                  1
Grumpier Old Men                                         2
Waiting to Exhale                                        3
Father of the Bride Part II                              4
                                                      ... 
The Last Brickmaker in America                        9214
Rustom                                                9215
Mohenjo Daro                                          9216
Shin Godzilla                                         9217
The Beatles: Eight Days a Week - The Touring Years    9218
Length: 9219, dtype: int64

now we will use get recommendation with updated cosine similarity

In [81]:
get_recommendations('The Dark Knight').head(10)

8031         The Dark Knight Rises
6218                 Batman Begins
6623                  The Prestige
2085                     Following
7648                     Inception
4145                      Insomnia
3381                       Memento
8613                  Interstellar
7659    Batman: Under the Red Hood
1134                Batman Returns
Name: title, dtype: object

from the above we can be satisfied since the system recognise the Chistopher Nolan Movies. we can also experiment on this engine by giving more weights to Directors, cast,geners. by reducing the importance or weightage given to the Keywords by limiting the keywords.weighing geners based on their freaquency

In [82]:
get_recommendations('Mean Girls').head(10)

3319               Head Over Heels
4763                 Freaky Friday
1329              The House of Yes
6277              Just Like Heaven
7905         Mr. Popper's Penguins
7332    Ghosts of Girlfriends Past
6959     The Spiderwick Chronicles
8883                      The DUFF
6698         It's a Boy Girl Thing
7377       I Love You, Beth Cooper
Name: title, dtype: object

From the above we can see that Batman: Under the Red Hood is similar to Darknight Raises because of the similar characters in both films. But Batman: Under the Red Hood is not a good movie. when commpared to other Batman Movies. lets work on this detailings

In [83]:
idx = indices['Toy Story']

In [84]:
sim_score = list(enumerate(cos_sim[idx]))

In [85]:
sim_score = sorted(sim_score,key= lambda x: x[1],reverse=True)

In [86]:
sim_score = sim_score[1:20]
movie_ind = [i[0] for i in sim_score]
movi = smd.iloc[movie_ind][['title','vote_average','vote_count','year']]
vote_count = movi[movi['vote_count'].notnull()]['vote_count'].astype('int')
vote_ava = movi[movi['vote_average'].notnull()]['vote_average'].astype('int')


In [87]:
c = vote_ava.mean()
m = vote_count.quantile(0.60)
quantifi = movi[(movi['vote_count']>= m) & (movi['vote_count'].notnull()) & (movi['vote_average'].notnull())]
quantifi['vote_count'] = quantifi['vote_count'].astype('int')
quantifi['vote_average'] = quantifi['vote_average'].astype('int')
quantifi['wr'] =  quantifi.apply(weig_rat, axis=1)
quantifi =quantifi.sort_values('wr', ascending=False).head(10)
print(quantifi)

                                   title  vote_average  vote_count  year  \
7648                           Inception             8       14075  2010   
8613                        Interstellar             8       11187  2014   
6623                        The Prestige             8        4510  2006   
3381                             Memento             8        4168  2000   
8031               The Dark Knight Rises             7        9263  2012   
6218                       Batman Begins             7        7511  2005   
1134                      Batman Returns             6        1706  1992   
9024  Batman v Superman: Dawn of Justice             5        7189  2016   

            wr  
7648  7.710253  
8613  7.645088  
6623  7.260651  
3381  7.217222  
8031  6.734063  
6218  6.683227  
1134  5.628269  
9024  5.045810  


<ipython-input-87-5946e57b411c>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  quantifi['vote_count'] = quantifi['vote_count'].astype('int')
<ipython-input-87-5946e57b411c>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  quantifi['vote_average'] = quantifi['vote_average'].astype('int')
<ipython-input-87-5946e57b411c>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: 

now we can create a funtion such that it accepts the argument of differnt movie and go thorught  the  series of pipeline to get the resullt

In [131]:
def final_reco(movi):
    idx = indices[movi]
    sim_score = list(enumerate(cos_sim[idx]))
    sim_score = sorted(sim_score,key= lambda x: x[1],reverse=True)
    sim_score = sim_score[1:20]
    movie_ind = [i[0] for i in sim_score]
    movi = smd.iloc[movie_ind][['title','vote_average','vote_count','year']]
    vote_count = movi[movi['vote_count'].notnull()]['vote_count'].astype('int')
    vote_ava = movi[movi['vote_average'].notnull()]['vote_average'].astype('int')
    c = vote_ava.mean()
    m = vote_count.quantile(0.60)
    quantifi = movi[(movi['vote_count']>= m) & (movi['vote_count'].notnull()) & (movi['vote_average'].notnull())]
    quantifi['vote_count'] = quantifi['vote_count'].astype('int')
    quantifi['vote_average'] = quantifi['vote_average'].astype('int')
    quantifi['wr'] =  quantifi.apply(weig_rat, axis=1)
    quantifi =quantifi.sort_values('wr', ascending=False).head(10)
    return quantifi

In [132]:
final_reco('Mean Girls')

<ipython-input-131-24f89e4575b5>:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  quantifi['vote_count'] = quantifi['vote_count'].astype('int')
<ipython-input-131-24f89e4575b5>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  quantifi['vote_average'] = quantifi['vote_average'].astype('int')
<ipython-input-131-24f89e4575b5>:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documenta

,title,vote_average,vote_count,year,wr
1547,The Breakfast Club,7,2189,1985,6.709602
8883,The DUFF,6,1372,2015,5.818541
3712,The Princess Diaries,6,1063,2001,5.781086
4763,Freaky Friday,6,919,2003,5.757786
6277,Just Like Heaven,6,595,2005,5.681521
6959,The Spiderwick Chronicles,6,593,2008,5.680901
7332,Ghosts of Girlfriends Past,5,716,2009,5.092422
7905,Mr. Popper's Penguins,5,775,2011,5.087912
